In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import r2_score
import os, glob


from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# HOW CAN I GET LAGED FEATURES??????

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [ ]:
def realized_volatility_per_time_id(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock, int(stock_id)

In [ ]:
def past_realized_volatility_per_stock(list_file, train_df, prediction_column_name, train=True, models=None):
    df_past_realized = pd.DataFrame()
    models_arr = []
    for file in list_file:
        df_realized_vol_per_stock, stock_id = realized_volatility_per_time_id(file, prediction_column_name)
        test_stock = train_df.query('stock_id == @stock_id')
        if train:
            
            test_stock_full, model = get_features_and_model(test_stock, df_realized_vol_per_stock)
            models_arr.append(model)
            df_past_realized = pd.concat([df_past_realized, test_stock_full])
            
        elif models:
            test_stock_full = get_features_and_model(test_stock, df_realized_vol_per_stock, 
                                                     train=False, model=models[stock_id])
            df_past_realized = pd.concat([df_past_realized, test_stock_full])
            
    return df_past_realized, models_arr

In [ ]:
def fitting_model(test_stock_full):
    
    
    X = test_stock_full[test_stock_full.columns[5:].tolist()]
    y = test_stock_full['target']
    model = XGBRegressor(n_estimators=500, max_depth=10, eta=0.1, subsample=0.7, colsample_bytree=0.8)

    model.fit(X, y)
    test_stock_full['boost_pred'] = model.predict(X)
    get_score(test_stock_full,'boost_pred') 
    return test_stock_full, model

In [ ]:
def get_features_and_model(test_stock, df_past_realized_train, train=True, model=None):
        
    # features
    test_stock.loc[:,'diff'] = test_stock['target'].diff(2).values
    for i in range(2,7):
        test_stock.loc[:, 'laged_' + str(i)]  = test_stock.target.shift(i).values
    
    
    laged_col = ['laged_' + str(i) for i in range(2,7)]
    test_stock.loc[:, ['mean', 'std']] = test_stock[laged_col].agg(['mean','std'], axis=1).values
    test_stock_full = test_stock.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')
    
    if train:
        # fit model
        test_stock_full.dropna(inplace=True)
        test_stock_full, model = fitting_model(test_stock_full)
        return test_stock_full, model
    elif model:
        X = test_stock_full[test_stock_full.columns[5:].tolist()]
        test_stock_full['boost_pred'] = model.predict(X)
        return test_stock_full
    else:
        print('Some problems!')
        return None

In [ ]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def get_score(df,pred_col):
    R2 = round(r2_score(y_true = df['target'], y_pred = df[pred_col]),3)
    RMSPE = round(rmspe(y_true = df['target'], y_pred = df[pred_col]),3)
    print(f'Performance of the column {pred_col} : R2 score: {R2}, RMSPE: {RMSPE}')

In [ ]:
list_order_book_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*')
train = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)

In [ ]:
# test_stock = train.query('stock_id == 0')
# print(test_stock.shape)
# test_stock.reset_index(inplace=True)

In [ ]:
sns.displot(test_stock['target'])
plt.show()
sns.lineplot(x='index',y='target', data=test_stock)
plt.show()

In [ ]:
# ACF FEATURES


# sm.tsa.acf(test_stock['target'], nlags=3)[1:]
# sm.tsa.acf(test_stock['diff'], nlags=3, missing='drop')[1:]

# test_stock_full, model = get_features_and_model(test_stock, df_past_realized_train)

In [ ]:
df_past_realized_train, models = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                                    train_df = train,
                                                           prediction_column_name='pred')
df_past_realized_train.head()

In [ ]:
# get_score(test_stock_full,'pred') 
# get_score(test_stock_full,'laged_2') 
# get_score(test_stock_full,'boost_pred') 

In [ ]:
import pickle
with open('models.pkl', 'wb') as pickle_file:
    pickle.dump(models, pickle_file)

In [ ]:
test = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/test.csv')
test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)

In [ ]:
list_order_book_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')
df_naive_pred_test, models_arr = past_realized_volatility_per_stock(list_file=list_order_book_file_test, train_df=test,
                                                           prediction_column_name='target', train=False, models=models)

In [ ]:
test

In [ ]:
if df_naive_pred_test[0].empty:
    test['target'] = 0
    test.to_csv('submission.csv',index = False)
    print('Empty!')
else:
    df_naive_pred_test.to_csv('submission.csv',index = False)